### Load Packages

In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
from tqdm import tqdm
import tensorflow as tf
import matplotlib.pyplot as plt
tqdm.pandas()
np.set_printoptions(5,)
assert int(tf.__version__[0]) == 2, "tensorflow 2.0 should be installed"

/opt/conda/lib/python3.6/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


### Download Data

In [2]:
from tensorflow.keras.utils import get_file

ROOT_URL = "https://craftsangjae.s3.ap-northeast-2.amazonaws.com/data/"

# 데이터 가져오기
play_path = get_file("lastfm_play.csv",
                     ROOT_URL+"lastfm_play.csv")
artist_path = get_file("lastfm_artist.csv",
                       ROOT_URL+"lastfm_artist.csv")
user_path = get_file("lastfm_user.csv",
                     ROOT_URL+"lastfm_user.csv")

play_df = pd.read_csv(play_path)
artist_df = pd.read_csv(artist_path)
user_df = pd.read_csv(user_path)

8036352/8033280 [==============================] - 2s 0us/step


### Load Bayesian Personalized Ranking

In [6]:
from tensorflow.keras.models import load_model

fpath = get_file("bpr_lastfm.h5", 
                 "https://craftsangjae.s3.ap-northeast-2.amazonaws.com/models/bayesian_personalized_ranking_lastfm.h5")
model = load_model(fpath, custom_objects={""})

ValueError: Unknown loss function:log_sigmoid_loss

In [5]:
# Extract Embedding weights
user_embeddings = model.get_layer('user_embedding').get_weights()[0]
item_embeddings = model.get_layer('item_embedding').get_weights()[0]
item_bias = model.get_layer('item_bias').get_weights()[0]

# Convert numpy array to Dataframe
user_embedding_df = pd.DataFrame(user_embeddings, 
                                 index=user_df.user_id)
user_embedding_df[num_factor] = 1.

artist_embedding_df = pd.DataFrame(item_embeddings,
                                   index=artist_df.artist_name)
artist_embedding_df[num_factor] = item_bias[:,0]

NameError: name 'model' is not defined

# Matrix Factorization Using Annoy
----

## 1. Build Annoy Tree

In [ ]:
from annoy import AnnoyIndex

tree = AnnoyIndex(33, "dot") # Dot 유사도를 이용하기

for idx, value in enumerate(artist_embedding_df.values):
    tree.add_item(idx, value)
    
tree.build(20) 

## 2. Search Similarity using Annoy


**메탈, 락과 같은 음악을 많은 들은 사람**


In [ ]:
target_id = 300
target_user = user_embedding_df.loc[target_id]
target_user.loc[num_factor] = 1
print([artist_id2name[f] for f in artistset_per_user[target_id]])

Annoy를 통해 검색하려면 아래와 같이 작성하면 됩니다.

In [ ]:
%%time
artist_indices = tree.get_nns_by_vector(target_user, 10)
artist_indices

Annoy를 통해 검색하게 되면 훨씬 더 빠른 속도 내에 처리할 수 있습니다. 현재 컴퓨터 성능으로는 대략 200배 정도 빨라집니다.

이 인덱스를 아티스트 이름으로 바꾸면 아래와 같게 됩니다.

In [ ]:
artist_embedding_df.index[artist_indices]

**힙합을 많이 들은 사람**

In [ ]:
target_id = 1200
target_user = user_embedding_df.loc[target_id]
target_user.loc[num_factor] = 1
print([artist_id2name[f] for f in artistset_per_user[target_id]])

Annoy를 통해 검색하면 아래와 같습니다.

In [ ]:
artist_indices = tree.get_nns_by_vector(target_user, 10)

이 인덱스를 아티스트 이름으로 바꾸면 아래와 같습니다.

In [ ]:
artist_embedding_df.index[artist_indices]

**댄스 음악을 좋아하는 사람**

In [ ]:
target_id = 209
target_user = user_embedding_df.loc[target_id]
target_user.loc[num_factor] = 1
print([artist_id2name[f] for f in artistset_per_user[target_id]])

Annoy를 통해 검색하면 아래와 같습니다.

In [ ]:
artist_indices = tree.get_nns_by_vector(target_user, 10)

이 인덱스를 아티스트 이름으로 바꾸면 아래와 같습니다.

In [ ]:
artist_embedding_df.index[artist_indices]